# Kaggle Titanic


In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [ ]:
ls -1ha kaggle.json

ls: cannot access 'kaggle.json': No such file or directory


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# permission warning이 일어나지 않도록
!chmod 600 ~/.kaggle/kaggle.json

# 내가 참가한 모든 대회보기
!kaggle competitions list


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!kaggle competitions download -c titanic

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!ls

sample_data


In [ ]:
import numpy as np
import pandas as pd
!pip3 install matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
sns.set(font_scale = 2.5)
# matplotlib의 기본 scheme 말고 seaborn scheme을 세팅하고, 일일이 graph의 font_size를 지정할 필요없이 seaborn의 font_scale을 사용하면 편하다.

import missingno as msno # 결측치 시각화

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
# 프로세스
# 데이터셋 확인 -> 탐색적 데이터 분석 -> feature engineering -> 모델 만들기 -> 모델 학습 및 예측 -> 모델 평가

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_test.head()
# Pclass : 티켓의 클래스 ( Passenger Class)
# SibSp = 함께 탑승한 형제와 배우자의 수
# cabin : 객실번호
# embarked : 탑승 항구

FileNotFoundError: ignored

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

In [ ]:
for col in df_train.columns:
  msg = 'column: {:>10}\t Percent of Nan Value : {:.2f}%'.format(col,100*(df_train[col]).isnull().sum()/df_train[col].shape[0])
  # {:.2f}는 두번째 자리까지 표현하는 것을 뜻한다.
  print(msg)

In [ ]:
for col in df_test.columns:
  # shape[0] 의 갯수는 전체 행의 개수
  # shape[1] 의 갯수는 전체 열의 갯수
  #print(df_test[col].shape[0])
  msg = 'column: {:>10}\t Percent of NaN Values : {:.2f}%'.format(col,100*(df_test[col]).isnull().sum()/df_test[col].shape[0])
  print(msg)

In [ ]:
msno.matrix(df= df_train.iloc[:,:],figsize=(8,8),color = (0.8,0.5,0.2))
# 흰색으로 된 빈칸들이 결측치이다.
# color의 3가지는 RGB이다.


# iloc : [행,열]
# 여기서는 모든 행과 열에 대해서 msno를 파악하고 있음

In [ ]:
msno.bar(df=df_train.iloc[:,:],figsize=(8,8),color=(0.8,0.5,0.2))

# cabin은 결측치가 4/5 이상으로 대부분이 결측치임을 확인할 수 있다.


In [ ]:
msno.bar(df=df_test.iloc[:,:],figsize = (10,10), color = (0.2,0.75,0.05))

# Target Label 확인


In [ ]:
f, ax = plt.subplots(1,2,figsize=(18,8))

df_train['Survived'].value_counts().plot.pie(explode=[0,0.3], autopct = '%1.1f%%',ax=ax[0],shadow=True, startangle= 75)
# ax는 그래프가 어디에 위치할지를 결정한다.
# autopct = 각 범주가 데이터에서 차지하는 비율
# startangle = pie의 시작각도
ax[0].set_title('Pie plot - Survived')
ax[0].set_ylabel('')

sns.countplot('Survived',data=df_train,ax=ax[1])
ax[1].set_title('Count plot - Survived')
# explode : 조각내기
# explode는 부채꼴이 파이 차트 중심에서 벗어나는 정도를 설정한다.

plt.show()

In [ ]:
df_train['Pclass']

In [ ]:
df_train[['Pclass','Survived']]
# 두개 이상의 데이터 베이스를 같이 연결하려면 이중 배열을 해야한다. 왜냐하면 두개를 합쳐줘야 하기 때문이다.

In [ ]:
df_train[['Pclass','Survived']].groupby(['Pclass'],as_index= True).count()
# count를 하면 각 class에 몇명에 있는지 확인 할 수 있다

In [ ]:
df_train[['Pclass','Survived']].groupby(['Pclass'],as_index= False).count()

In [ ]:
df_train[['Pclass','Survived']].groupby(['Pclass'],as_index= True).sum()

In [ ]:
pd.crosstab(df_train['Pclass'],df_train['Survived'],margins=True).style.background_gradient(cmap='autumn')

In [ ]:
df_train[['Pclass','Survived']].groupby(['Pclass'],as_index=True).mean().sort_values(by='Survived',ascending=False).plot.bar(color=(0.2,0.7,0.1))

In [ ]:
y_position = 1.02
f,ax = plt.subplots(1,2,figsize=(25,8)) # column 2개
df_train['Pclass'].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'],ax = ax[0])
ax[0].set_title('Number of Passenger by Pclass',y=y_position)
ax[0].set_ylabel('Count')
sns.countplot('Pclass',hue = 'Survived',data = df_train,ax=ax[1])
ax[1].set_title('Pclass : Survived vs Dead', y= y_position)
plt.show()


In [ ]:
f, ax = plt.subplots(1,2,figsize=(25,8))
df_train[['Sex','Survived']].groupby(['Sex'],as_index = True).mean().plot.bar(ax=ax[0],cmap='summer')
ax[0].set_title('Survived vs Sex')
sns.countplot('Sex',hue='Survived',data =df_train,ax=ax[1]) # hue는 카테고리 이름을 넣어주는 것
ax[1].set_title('Sex : Survived vs Dead')
plt.show()

In [ ]:
df_train[['Sex','Survived']].groupby(['Sex'],as_index=False).mean().sort_values(by='Survived',ascending=True)
# 1이 생존했음을 뜻하고, 0 이 죽었을 확률이니까 mean값이 살아있을 확률이 될수 있는 것이다

In [ ]:
pd.crosstab(df_train['Sex'],df_train['Survived'],margins=True).style.background_gradient(cmap='summer_r')
# margins=True 옵션을 설정해주면 행과 열을 기준으로 합계를 같이 제시해줘서 꽤 편리하다.

# Both Sex and Pclass

In [ ]:
sns.factorplot('Pclass','Survived',hue='Sex',data = df_train, size=6,aspect=1.5)
# 3개 차원으로 이뤄진 그래프를 그릴 수 있다

In [ ]:
sns.factorplot(x = 'Sex',y='Survived',col='Pclass',data = df_train,size=9,aspect=1)
# saturation _ 채도 :

In [ ]:
sns.factorplot(x = 'Sex',y='Survived',col='Pclass',data = df_train,size=9,satureation=.5,aspect=1)

In [ ]:
# age

print('제일 내일 나이 많은 탐승객 : {:.2f} Years'.format(df_train['Age'].max()))
print("제일 어린 탑승객 : {:.2f} Years".format(df_train['Age'].min()))
print("탑승객 평균 나이 : {:.2f} Years".format(df_train['Age'].mean()))

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(9,5))
import seaborn as sns
sns.kdeplot(df_train[df_train['Survived'] == 1]['Age'], ax=ax)
sns.kdeplot(df_train[df_train['Survived']==0]['Age'],ax=ax)
plt.legend(['Survived==1','Survived==0'],loc = 'lower right')
plt.show()

In [ ]:
# Age Distribution withing classes
plt.figure(figsize=(8,6))
df_train['Age'][df_train['Pclass']==1].plot(kind='kde')
df_train['Age'][df_train['Pclass']==2].plot(kind='kde')
df_train['Age'][df_train['Pclass']==3].plot(kind='kde')

plt.xlabel('Age')
plt.title('Age Distribution within Classes')
plt.legend(['1st Class', '2nd Class', '3rd Class'])

In [ ]:
cummulate_survival_ratio = []
for i in range(1,80):
  cummulate_survival_ratio.append(df_train[df_train['Age']<i]['Survived'].sum()/len(df_train[df_train['Age']<i]['Survived']))

plt.figure(figsize=(7,7))
plt.plot(cummulate_survival_ratio)
plt.title('Survival rate Change depending on range of Age',y=1.02) # 제목부터 도표까지 indent
plt.ylabel('Survival Rate')
plt.xlabel('Range of Age(0~x)')
plt.show

In [ ]:
f,ax = plt.subplots(1,2,figsize=(18,8))
sns.violinplot('Pclass','Age',hue= 'Survived',data = df_train,scale='count', split = True,ax=ax[0]) # hue는 카테고리 이름을 넣어주는 것. ( legend와 같은 뜻이다)
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0,110,10)) # 눈금
sns.violinplot('Sex','Age', hue = 'Survived', data = df_train, scale ='count',split = True, ax=ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0,110,10))
plt.show()

In [ ]:
 f , ax = plt.subplots(1,1,figsize=(7,7))
df_train[['Embarked','Survived']].groupby(['Embarked'],as_index = True).mean().sort_values(by='Survived',ascending = False).plot.bar(ax=ax)
# as_index를 해야 Embarked를 index로 받을 수 있다.

In [ ]:
f, ax = plt.subplots(2,2, figsize = (20,15))
sns.countplot('Embarked',data = df_train, ax=ax[0,0])
ax[0,0].set_title('(1) No. Of Passengers Boarded' )
sns.countplot('Embarked',hue = 'Sex', data = df_train,ax=ax[0,1])
ax[0,1].set_title('(2) Male-Female Split for Embarked')
sns.countplot('Embarked',hue = 'Survived', data = df_train, ax= ax[1,0])
ax[1,0].set_title('(3) Embakred vs Survived')
sns.countplot('Embarked', hue='Pclass', data=df_train, ax=ax[1,1])
ax[1,1].set_title('(4) Embarked vs Pclass')
plt.subplots_adjust(wspace = 0.2, hspace = 0.5) # wspace : column 간 사이의 거리, hspace : row 간의 거리



In [ ]:
df_train['FamilySize'] = df_train['SibSp']+df_train['Parch'] +1 # 자기자신을 더하면 가족이됩니다.
# 함께 탑승한 부모, 아이의 수
df_test['FamilySize'] = df_test['SibSp']+df_test['Parch']+1 # 자기자신을 포함해야하니까 1을 더한다.


In [ ]:
print("Maximum size of Family",df_train['FamilySize'].max())
print("Minimum size of Family", df_train['FamilySize'].min())

In [ ]:
f, ax = plt.subplots(1,3,figsize=(40,10))
sns.countplot('FamilySize', data =df_train, ax= ax[0])
ax[0].set_title('(1) No. Of passengers Boarded',y = 1.02)

sns.countplot('FamilySize',hue = 'Survived',data = df_train,ax=ax[1])
ax[1].set_title('(2) Survived Countplot Depending on FamilySize',y  = 1.02)

df_train[['FamilySize','Survived']].groupby(['FamilySize'],as_index= True).mean().sort_values(by='Survived',ascending = False).plot.bar(ax=ax[2])

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8))
g = sns.distplot(df_train['Fare'],color='b',label='Skewness : {:.2f}'.format(df_train['Fare'].skew(),ax=ax))

g = g.legend(loc = 'best')

# 굉장히 치우쳐져 있는 distribution을 관찰 할 수 있다. 
# 이대로 모델에 넣어주면 모델이 잘못 학습할 수 있다.
# outlier의 영향을 줄이기 위해 fare에 log를 취할 생각이다.
# dataframe의 특정 colum에 공통된 작업을 적용하고 싶으면 map, apply를 사용하면 손쉽게 적용할수 있다.
# fare columns의 데이터 모두를 log값 취하는것이 필요한데, 파이썬의 간단한 lambda 함수를 이요해 간단한 로그를 적용하는 함수를 map에 인수로 넣어주면 fare columns 데이터에 그대로 적용이 된다.

In [ ]:
df_test.loc[df_test['Fare'].isnull(),'Fare'] = df_test['Fare'].mean()  # testset에 있는 nan value를 평균값으로 치환한다.
# df.loc['row, 'column'] 로 쓴다
# df.iloc['row name ' , 'column name']

df_train['Fare'] = df_train['Fare'].map(lambda i : np.log(i) if i>0 else 0)
df_test['Fare']= df_test['Fare'].map(lambda i : np.log(i) if i>0 else 0)

In [ ]:
# df = pd.DataFrame(np.arange(10, 22).reshape(3, 4),
#                   index=["a", "b", "c"],
#                   columns=["A", "B", "C", "D"])
# df['EmPTY 1']= ''
# df


In [ ]:

# df.loc[df.B.isnull(),'Empty1'] = df['B'].mean()
# df

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,8))
g = sns.distplot(df_train['Fare'],color = 'b',label='Skewness : {:.2f}'.format(df_train['Fare'].skew(),ax=ax))

g = g.legend(loc='best')

In [ ]:
df_train.head()

In [ ]:
df_train['Ticket'].value_counts()

In [ ]:
df_train.isnull().sum() # age와 cabin, embarked에 각각 null값이 있음을 확인할 수 있다